In [1]:
from PIL import Image
from create_dataset import create_coco, create_object2color

import matplotlib.pyplot as plt
import os
import numpy as np
import tqdm
import datetime
import json
%matplotlib inline

label_name = os.listdir('./Dataset/sample_train_label/')
label_path = ['./Dataset/sample_train_label/'+name for name in label_name]
label_path.sort()
############# To Delete ###############
label_path = label_path
############# To Delete ###############

img_name = os.listdir('./Dataset/sample_train_color/')
img_path = ['./Dataset/sample_train_color/'+name for name in img_name]
img_path.sort()
############# To Delete #1#############
img_path = img_path
############# To Delete ###############

In [3]:
object2color = np.load('./Dataset/object2color.npy')
create_coco(img_path, label_path, './Dataset/data.json', object2color=object2color, crop_size=[75, 75, 28, 28], down_scale=4)
#create_coco(img_path, label_path, './Dataset/data.json')

Creating image list...
object2color is obtained...


UnboundLocalError: local variable 'class_instance' referenced before assignment

In [12]:
object2color.any()

{(1, 0): (82, 7, 198),
 (1, 1): (82, 2, 26),
 (1, 2): (221, 144, 33),
 (1, 3): (251, 136, 60),
 (1, 4): (78, 92, 137),
 (1, 5): (83, 239, 115),
 (1, 6): (50, 0, 184),
 (1, 7): (232, 37, 75),
 (1, 8): (143, 232, 227),
 (1, 9): (142, 176, 21),
 (1, 10): (154, 48, 171),
 (1, 11): (123, 207, 243),
 (1, 12): (212, 121, 73),
 (1, 13): (184, 119, 218),
 (1, 14): (31, 218, 191),
 (1, 15): (73, 209, 51),
 (1, 16): (170, 237, 197),
 (1, 17): (241, 1, 76),
 (1, 18): (117, 199, 115),
 (1, 19): (247, 174, 254),
 (1, 20): (149, 75, 189),
 (1, 21): (194, 39, 174),
 (1, 22): (133, 161, 207),
 (1, 23): (120, 1, 223),
 (1, 24): (200, 7, 237),
 (2, 0): (47, 174, 244),
 (2, 1): (205, 206, 237),
 (2, 2): (171, 160, 32),
 (2, 3): (100, 4, 120),
 (2, 4): (188, 145, 111),
 (3, 0): (245, 80, 36),
 (3, 1): (157, 25, 1),
 (3, 2): (250, 101, 132),
 (3, 3): (164, 68, 50),
 (3, 4): (180, 119, 132),
 (3, 5): (35, 193, 68),
 (4, 0): (37, 116, 252),
 (4, 1): (100, 53, 19),
 (4, 2): (201, 71, 139),
 (4, 3): (242, 105, 